# Managing Azure Machine Learning Workspace Connections

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create Azure Machine Learning workspace Connection from Python SDK

**Motivations** - Azure Machine Learning Workspace Connection provides secure and trustworthy way to access external resources from AML.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the compute will be created.

## 1.1. Import the required libraries

In [ ]:
# import required libraries
import os
from azure.ai.ml import MLClient
from azure.ai.ml.entities import WorkspaceConnection
from azure.identity import DefaultAzureCredential

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
# Enter details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
# get a handle to the workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

# 2 Create Workspace Connection
An Azure Machine Learning workspace is secure and trustworthy way for AML users to store credentials to access external resources from AML

Workspace Connection can be created using the `WorkspaceConnection` class. This class accepts the following key parameters:
- `name` - Name of connection.
- `target` - The URL or ARM resource ID of the external resource.
- `credentials` - The credentials for authenticating to the external resource.
- `type` - The category of external resource for this connection.

## 2.1 Create a Workspace Connection
In this example we will create a **Workspace Connection** to an [**Azure Container Registry**]([ACR](https://azure.microsoft.com/en-us/products/container-registry/)) and use **username/password** credentials

Once connection is successfully created you can reference images from [Azure Container Registry]([ACR](https://azure.microsoft.com/en-us/products/container-registry/)) in AML environment and AML will automatically fetch these credentials while building the environment.

In [ ]:
from azure.ai.ml.entities import WorkspaceConnection
from azure.ai.ml.entities import UsernamePasswordConfiguration

# fetching secrets from env var to secure access, these secrets can be set outside or source code
acr_username = os.environ["ACR_USERNAME"]
acr_password = os.environ["ACR_PASSWORD"]

credentials = UsernamePasswordConfiguration(
    username=acr_username, password=acr_password
)

ws_connection = WorkspaceConnection(
    name="<connection_name>",
    target="<acr_url>",  # testacr.azurecr.io,
    type="container_registry",
    credentials=credentials,
)

ml_client.connections.create_or_update(ws_connection)

## 2.2 Create a Workspace Connection for private feeds
In this example we will create a **Workspace Connection** to **private git feed** and use **Personal Access Token (PAT)** credentials

Once connection is successfully created you can reference feeds from **private git feed** in AML environment and AML will automatically fetch these credentials while building the environment.

In [ ]:
from azure.ai.ml.entities import WorkspaceConnection
from azure.ai.ml.entities import PatTokenConfiguration

# fetching secrets from env var to secure access, these secrets can be set outside or source code
git_pat = os.environ["GIT_PAT"]

credentials = PatTokenConfiguration(pat=git_pat)

ws_connection = WorkspaceConnection(
    name="<connection_name>", target="<git_url>", type="git", credentials=credentials
)

ml_client.connections.create_or_update(ws_connection)

## 2.3 Create a Workspace Connection for private python feeds
In this example we will create a **Workspace Connection** to **private python feed** and use **SAS Token** credentials

Once connection is successfully created you can reference feeds from **private python feed** in AML environment and AML will automatically fetch these credentials while building the environment.

In [ ]:
from azure.ai.ml.entities import WorkspaceConnection
from azure.ai.ml.entities import SasTokenConfiguration

# fetching secrets from env var to secure access, these secrets can be set outside or source code
python_feed_sas = os.environ["PYTHON_FEED_SAS"]

credentials = SasTokenConfiguration(sas_token=python_feed_sas)

ws_connection = WorkspaceConnection(
    name="<connection_name>",
    target="<python_feed_url>",
    type="python_feed",
    credentials=credentials,
)

ml_client.connections.create_or_update(ws_connection)